# Cleaning training data

In [12]:
import pandas as pd
import csv

delimiter = '§'
data_dict = {}
dataset_folder = 'traduction_occitan/'
filename = 'train.csv'
file_path = dataset_folder + filename

# Open the CSV file
with open(file_path, 'r') as file:
    csv_reader = csv.reader(file, delimiter=delimiter, quotechar=None)
    for i, row in enumerate(csv_reader):
        # print(row)
        # print()
        key = row[2]
        if key in data_dict:
            data_dict[key].append([row[0], row[1]])
        else:
            data_dict[key] = [[row[0], row[1]]]
        if len(row) != 3:
            print(f'{len(row)=} ; line:{i} ; row:{row}\n\n')

In [13]:
import Levenshtein
ex = data_dict['lengadoc'][5]
print(Levenshtein.distance(ex[0], ex[1]))
print(ex[0])
print(ex[1])

1
Patrici Baccou
Patrice Baccou


In [14]:
data_dict.keys()

dict_keys(['lengadoc', 'gascon'])

In [15]:
# check ligne foireuse
def check_lignes_duplique():
    for key in data_dict.keys():
        dataset = data_dict[key]
        count = 0
        for example in dataset:
            dist = Levenshtein.distance(example[0], example[1])
            if dist == 0:
                count += 1
                # print(example)
                # if count >= 10:
                #     break
        print(f'erreurs : {count}, sur {len(dataset)} examples ({round((count/len(dataset)) * 100, 2)}%)')
check_lignes_duplique()

erreurs : 10, sur 282 examples (3.55%)
erreurs : 61, sur 2718 examples (2.24%)


In [16]:
# clean lignes foireuses
for key in data_dict.keys():
    dataset = data_dict[key]
    data_dict[key] = [[a, b] for a, b in dataset if Levenshtein.distance(a, b) != 0]

In [17]:
check_lignes_duplique()

erreurs : 0, sur 272 examples (0.0%)
erreurs : 0, sur 2657 examples (0.0%)


In [18]:
# PART 1 : Préparation de `data_dict` (déjà fourni dans votre code)
# ...

# PART 2 : Création d'un nouveau fichier CSV à partir de `data_dict`
output_file_path = 'traduction_occitan/' + filename.split('.')[0] + 'Norm.csv'

# Ouvrir le fichier de sortie
with open(output_file_path, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    
    # Itérer sur chaque clé (dialecte) dans `data_dict`
    for key in data_dict.keys():
        dataset = data_dict[key]  # Liste de paires [occitan, français]
        
        # Écrire chaque paire dans le fichier CSV
        for occitan, french in dataset:
            writer.writerow([occitan, french, key])

print("La création du fichier CSV à partir de data_dict est terminée. Le fichier de sortie est :", output_file_path)


La création du fichier CSV à partir de data_dict est terminée. Le fichier de sortie est : traduction_occitan/testNorm.csv


# More cleaning

In [22]:
import csv
from Levenshtein import distance

input_file_path = 'traduction_occitan/' + filename.split('.')[0] + 'Norm.csv'


# Fonction pour vérifier s'il y a des doublons dans une ligne
def verifier_doublons(csv_path):
    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for i, row in enumerate(reader):
            # Assurez-vous que la ligne a au moins deux colonnes
            if len(row) >= 2:
                # Comparer le contenu des deux premières colonnes
                if row[0].strip().lower() == row[1].strip().lower():
                    print(f"Doublon trouvé à la ligne {i+1}: {row[0]} | {row[1]}")
                elif distance(row[0].strip().lower(), row[1].strip().lower()) == 0:  # Utiliser la distance de Levenshtein si nécessaire
                    print(f"Doublon (par distance) trouvé à la ligne {i+1}: {row[0]} | {row[1]}")
            else:
                print(f"Ligne {i+1} ignorée (pas assez de colonnes).")

verifier_doublons(input_file_path)

Doublon trouvé à la ligne 6082: los òmes ! | Los òmes !
Doublon trouvé à la ligne 6087: los òmes ! | Los òmes !
Doublon trouvé à la ligne 6947: Bona Annada 2020 ! | Bona annada 2020 !
Doublon trouvé à la ligne 12430: DVD  Festival Escambis 5ème édition 	 | DVD  Festival Escambis 5ème édition
Doublon trouvé à la ligne 22476: Ofici public de la lengua occitana 22, bd maréchal juin 31406 Tolosa cedex 9 Tel 05 82 60 02 80 | Ofici public de la lengua occitana 22, bd Maréchal Juin 31406 Tolosa cedex 9 Tel 05 82 60 02 80
Doublon trouvé à la ligne 27940: La lenga que parlam es patrimòni de l'umanitat ! | la lenga que parlam es patrimòni de l'umanitat !
Doublon trouvé à la ligne 28903: monsieur, je crois que j'ai dit une grosse connerie ! | Monsieur, je crois que j'ai dit une grosse connerie !
Doublon trouvé à la ligne 38505: dragon | Dragon


In [23]:
import csv

input_file_path = 'traduction_occitan/trainNorm.csv'
occitan_output_file_path = 'traduction_occitan/occitan_train.csv'
french_output_file_path = 'traduction_occitan/french_train.csv'

# Ouvrir le fichier d'entrée
with open(input_file_path, 'r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    
    # Ouvrir les fichiers de sortie pour occitan et français
    with open(occitan_output_file_path, 'w', newline='', encoding='utf-8') as occitan_outfile, \
         open(french_output_file_path, 'w', newline='', encoding='utf-8') as french_outfile:
        
        occitan_writer = csv.writer(occitan_outfile)
        french_writer = csv.writer(french_outfile)
        
        # Lire chaque ligne du fichier d'entrée
        for row in reader:
            if len(row) < 2:  # S'assurer que la ligne contient les deux parties et le délimiteur
                continue
            # Écrire la partie occitane dans occitan_train.csv
            occitan_writer.writerow([row[0]])
            # Écrire la partie française dans french_train.csv
            french_writer.writerow([row[1]])  # Adjuster l'indice si nécessaire selon la structure exacte de vos données

print("La séparation est terminée. Les fichiers occitan_train.csv et french_train.csv ont été créés.")


La séparation est terminée. Les fichiers occitan_train.csv et french_train.csv ont été créés.
